# PASO 1 — Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

PASO 2 — Cargar todos los equipos automáticamente

In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
files = {
    "EIA": "Comparative_analysis_Competitive_strategies/data/EIA.xlsx",
    "HELLMEC": "C:/Users/Usuario/OneDrive - pascualbravo.edu.co/Documentos/GitHub/Comparative_analysis_Competitive_strategies/data/HELLMEC.xlsx",
    "KRATOS": "C:/Users/Usuario/OneDrive - pascualbravo.edu.co/Documentos/GitHub/Comparative_analysis_Competitive_strategies/data/KRATOS.xlsx",
    "OMEGA": "C:/Users/Usuario/OneDrive - pascualbravo.edu.co/Documentos/GitHub/Comparative_analysis_Competitive_strategies/data/OMEGA.xlsx",
    "PASCUAL": "C:/Users/Usuario/OneDrive - pascualbravo.edu.co/Documentos/GitHub/Comparative_analysis_Competitive_strategies/data/pascual bravo.xlsx",
    "SABANA": "C:/Users/Usuario/OneDrive - pascualbravo.edu.co/Documentos/GitHub/Comparative_analysis_Competitive_strategies/data/Sabana.xlsx",
    "VTECCI": "C:/Users/Usuario/OneDrive - pascualbravo.edu.co/Documentos/GitHub/Comparative_analysis_Competitive_strategies/data/VTECCI.xlsx",
}

#dfs = {}

file_path = os.path.join("..", "/content/data", "EIA.xlsx")

#for team, path in files.items():
#    print(f"Loading data for {team} from {path}")
df = pd.read_excel(file_path)
df.head()
    #df["team"] = team
    #dfs[team] = df

FileNotFoundError: [Errno 2] No such file or directory: '/content/data/EIA.xlsx'

PASO 3 — Limpieza y estandarización

In [ ]:
def clean_data(df):
    df = df.copy()
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    
    df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
    df = df.sort_values("timestamp")
    
    numeric_cols = ["car_velocity", "car_voltage", "car_current", "power"]
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    
    df = df.dropna(subset=["timestamp"])
    df["dt"] = df["timestamp"].diff().dt.total_seconds()
    df["dt"] = df["dt"].fillna(0)
    
    return df

for team in dfs:
    dfs[team] = clean_data(dfs[team])

PASO 4 — Cálculo de energía y eficiencia

In [ ]:
def compute_metrics(df):
    df = df.copy()
    
    # Energía en Wh
    df["energy_Wh"] = (df["power"] * df["dt"]) / 3600
    
    total_energy = df["energy_Wh"].sum()
    
    # Distancia
    if "car_trip" in df.columns and df["car_trip"].max() > 0:
        distance = df["car_trip"].max()
    else:
        df["distance_m"] = df["car_velocity"] * df["dt"]
        distance = df["distance_m"].sum() / 1000
    
    efficiency = total_energy / distance if distance > 0 else np.nan
    
    return {
        "Energy_Wh": total_energy,
        "Distance_km": distance,
        "Efficiency_Wh_per_km": efficiency,
        "Mean_Speed": df["car_velocity"].mean(),
        "Speed_std": df["car_velocity"].std(),
        "Power_mean": df["power"].mean(),
        "Power_p95": df["power"].quantile(0.95),
        "Coasting_ratio": np.mean(np.abs(df["power"]) < 5),
        "Regen_ratio": np.mean(df["power"] < 0)
    }

summary = []

for team, df in dfs.items():
    metrics = compute_metrics(df)
    metrics["Team"] = team
    summary.append(metrics)

summary_df = pd.DataFrame(summary)
summary_df = summary_df.sort_values("Efficiency_Wh_per_km")
summary_df

📌 Esta tabla es la principal del paper.

Guardar: